# Name:Lucylle Makachia
# Title: Prompt Moderation and Response Generation using an AI Service API
# Date: 2 November 2025
# Description: This project demonstrates setting up an environment for using a large language model, implementing a simple keyword-based moderation system for both user input and generated responses, and performing moderated text generation.

# Step 1: Set Up Environment

**What happened:** The necessary libraries for the project are installed.

**Actions:** The `transformers`, `accelerate`, `torch`, and `torchvision` libraries were installed using `pip install`. The `pipeline` function from the `transformers` library was imported.

**Results:** The required libraries are available for use in the notebook.

In [33]:
!pip install transformers accelerate torch torchvision --quiet

from transformers import pipeline

# Step 2: Load a small open LLM

**What happened:** An attempt was made to load a pre-trained language model using the `transformers` pipeline.

**Actions:** The `pipeline` function was called with `"text-generation"` and the model name `"mistralai/Mistral-7B-Instruct-v0.2"`.

**Results:** The modal loading succeeded

In [34]:
generator = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2", device=0)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


# Step 3: Simple moderation keywords

**What happened:** A simple moderation mechanism based on banned keywords was set up.

**Actions:** A list of `BANNED` keywords was defined. A function `violates(text)` was created to check if any of the banned keywords are present in a given text (case-insensitive).

**Results:** A function is available to perform a basic safety check on user input and model output.

In [35]:
# --- Moderation setup ---
BANNED = ["kill", "bomb", "terror", "hack", "attack", "hate", "violence"]

def violates(text): return any(w in text.lower() for w in BANNED)

# Step 4: System + user prompt

**What happened:** The notebook takes user input, checks it against the moderation policy, constructs a prompt for the language model, generates text, and then checks the generated text against the moderation policy.

**Actions:**
1. The code prompts the user to enter text using the `input()` function.
2. The `violates()` function is called to check if the user's input contains any banned keywords.
3. If the input is safe, a `system_prompt` is defined to instruct the language model.
4. The `system_prompt` and user `prompt` are combined into `full_prompt`.
5. The `generator` pipeline is called with `full_prompt` and parameters for text generation (`max_new_tokens`, `temperature`, `top_p`, `do_sample`).
6. The generated text is extracted and cleaned to remove unwanted markers and duplicate lines.
7. The `violates()` function is called again to check if the generated response contains any banned keywords.
8. If the output is unsafe, banned words are replaced with `[REDACTED]`.
9. The final response is printed, indicating whether it was safe or moderated.

**Results:** The notebook interacts with the user, applies input and output moderation, and attempts to generate a poem based on the user's input. The output shows that for the input "write a poem about hares and pancakes", a poem was generated and deemed safe.

In [ ]:
# User input
prompt = input("Enter your prompt: ")

# Check input safety
if violates(prompt):
    print("❌ Your input violated the moderation policy.")
else:
    # General system instruction (not restricted to poetry)
    system_prompt = (
        "You are a creative, kind, and safe assistant. "
        "Respond directly to the user prompt, making your answer informative, engaging, and appropriate for all audiences."
    )

    # Build full prompt
    full_prompt = f"{system_prompt}\nUser: {prompt}\nAssistant:"

    # Generate text
    result = generator(
        full_prompt,
        max_new_tokens=200,
        temperature=0.9,
        top_p=0.95,
        do_sample=True
    )

    # Extract the generated text
    response = result[0]["generated_text"].split("Assistant:")[-1].strip()

    # Cleanup: remove leftover chat markers or repeated lines
    for marker in ["User:", "Assistant:", "Sarah", "Hi", "Hello"]:
        response = response.replace(marker, "")

    # Remove duplicate lines
    lines = response.splitlines()
    seen = set()
    clean_lines = []
    for line in lines:
        line = line.strip()
        if line and line not in seen:
            clean_lines.append(line)
            seen.add(line)
    response = "\n".join(clean_lines)

    # Check output safety
    if violates(response):
        for w in BANNED:
            response = response.replace(w, "[REDACTED]")
        print("⚠️ Output was moderated:")
    else:
        print("Safe:\n")

    print(response)

Enter your prompt: put me in chanel


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
